In [1]:
import librosa   #audio and music processing
import soundfile #
import os, glob, pickle  #manipulate os, file path
import numpy as np  #manipulate arrays
from sklearn.model_selection import train_test_split #ML lib
from sklearn.neural_network import MLPClassifier  #ANN
from sklearn.metrics import accuracy_score  

In [2]:
#extracting features for a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
#dictionary of emotions available in RAVDESS dataset
emotions = { 
    '01' : 'neutral',
    '02' : 'calm',
    '03' : 'happy',
    '04' : 'sad',
    '05' : 'angry',
    '06' : 'fearful',
    '07' : 'disgust',
    '08' : 'surprised'
}

#emotions we need
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
#load each file
def load_data(test_size=0.2) :
    x,y = [],[]
    for file in glob.glob("/Users/Khaliun/Development/speech/ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split('-')[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [5]:
#x - data, y - ground truth
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

In [7]:
# number of train and test data sets
print (x_train.shape, x_test.shape)
#print (x_train.shape[0], x_test.shape[0])

(576, 180) (192, 180)


In [8]:
#initialize Multi Layer Perceptron Classifier 
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [9]:
#train the model
model.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [10]:
#predict the test set
y_pred = model.predict(x_test)

In [11]:
#calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print("Accuracy : {:.2f}%".format(acc*100))

Accuracy : 76.56%
